In [1]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# # Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
# login()

# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to("cuda") # or "cpu"

/home/kkj/ProtDiffusion/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/home/kkj/ProtDiffusion/.venv/lib/python3.10/site-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [10]:
# Generate a completion for a partial Carbonic Anhydrase (2vvb)
sequence = "MGSIQQDDVHNQIDHCNQSDDLPAARLNCNDVELFEVAGLACDETSSPTGMRDEMVLLSWLIALLRTREGGQIRYEWAYRYPEEEPVPRCLAMNEVVAGLQSSVKETAAAVSRHISADVSSPPAPASLLLSTSSLSQTSDEAKDEGLLHLEIAFENGLCKIRPTWHSENMLPFTVTRYARTLIDTVRLCISNCDAAIQDCLRPTAYDLDEIWRWNHNLPPTYNFCMHEIISDQAQKFPDKEAIASWDGSLTYRQIDQYSSFVARSLIGMGVGLHDVLPVCFEKSRWTIVAVLAVMKAGATFVLMDPTLPLARLQNMAQQVGAKMMVSSRGQYNLATEIIPNANVLVVEENTFSSLSAEQNGEPLPTVPSSALMYMIFTSGSTGTPKGVKISHETYTSSAIPRANAVGYTEDSRVLDFASYAFDVSIDSMLLTLGNGGCLCIPSDEDRLNDINGVIRRMKVNYAGLTPSVARILDADVISSLSGLGLGGEAVSARDVNLWGQDTRIIIGYGPCECTIGCTVNSSAATGRDYISIGPGNGAVIWIVDPNDHESLVPLGAVGELLVEGPIVGQGYLNDPEKTAAAFIEDPSWLVAGHEGYPGRRGRLYKTGDLGRYDPDGSGGIVFVGRKDTQVKLRGQRVELGEIESQLRARLPSETTVIAEVIVPQGSGGQPTLVAFVAAQTTKGHDHTGLEAAELPDELRRALSEADAELAKVLPRYMVPTAYIPVNHIPTLISGKTDRKRLRQFGATVDLRQLDQDATNTAARELSDLERRLRQAWSQTLKLQACSIRLQDNFFALGGDSLTAMKLVSVCRSQGLDLSVTSMFSNPTLSAMASVVRICDVDVQRTVPAFSMITSDMNSACVEAAEPCGVGPADIEDIYPCTPTQESLFTFSLKSVKPYVAQRVLCIPSHIDLNAWRKAWEDVVAALPILRTRVAQLQEPGLQQVVLKNSISWTQASDLAEYLENDRTQKMNLGESLARYAIVEDSADGKRYMVWTIHHVLYDGWSEPIILKQVSDALQGQPVEVKAQMRDFVRFVRDSDDAAVQEFWRRELKGAVGPQFPRLPSRDFMPTPDALVERQVSLDTSSGSPFTMATLIRGAWALVASQYTGSDDIVFGETLTGRDIPLPGVESIVGPLIATVPIRVRILRGSTVESYLQAVQQSVLARTPYQHLGMQNIRKVSQDAQHACETGTGLVIQPEPEYVGSELGVERGDVVLEALHFNPYPLMLACGIRKGGFRVCASFDSSLIETRQMERMLAQLETACWQLSQGLSRKVDEISCLPEAELNQIWQWNRSPPLSLDETTSRLRANASTKPGSSYPPAVVPWVCSPRNSSLLSPIGCVGELWLEGALLSGDTVDSPAWLVAGSSTCAGRTGKVQATGDMVQLREDGSLVFVGRKENVVPVQGHAVDITEIERHLAEHLPPTIRAAATVVRSSSDQELVMFIEQPAAEEACIELLSEKREIVCDAPDKAFQTTICATIPGSLAAVLKKLDKYMRDSLPSYMAPSAYIVVEKLPNTMDDIDHNLLNQIASQVTPQILNELRDGLSNAWTKATAPNHLSASESILRSAWAKVLRVDPEQIDVDDNFFRRGGDSVLAMKLVSSLRAQGYSLSVADIFRHMRLSDAARVMKVDERSTEKINSYQPFSMLRLPDVQQFLANIVRPQLGDQHWPIRDVLPVTDSQDMDIRATIQPPRTSIQYTMLYFDNSVDRERLFRSCSDLVKTHEILRTVFISHESSFLQVVLNELEIPVRAHKTDKQLDQYVASLFREDIESNFQLGCPFLRLFYVEGNNGESCLVIGLSHAQYDGVSLPRLLQDLDALYTGTQLATFSPFSLYMAQTSEEAIQNKAAAYWRNLLSSSSLSTLDGPSSDPTDKAIFHTRPVNIHPLKEITTANLLTAAWAMVLARRLQTPDVTFGSVTSGRTLDIPNAENFMGPCYQLTPVRVPFHPDWTASDLLNFVQTQSAESAAHDFLGFEKIAKLAGWASGRQGFDSIVHHQDWEDFDMMPFGGGSCRVDIANPHGDAAYPVKAVSFVKEGEIHVGVVCSERDVMFVDEVLGELAAAVVELAGQSTEVLLDSKLFSGQ"
length = 1024
divisor = 10

new_sequence = ""
for i, aa in enumerate(sequence):
    if i % divisor == 0:
        new_sequence += '_'
    else:
        new_sequence += aa
prompt = new_sequence[:length]
n_masked_aas = prompt.count('_')
n_aas = len(prompt) - n_masked_aas

In [ ]:
# Generate the sequence.
protein = ESMProtein(sequence=prompt)
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=10, temperature=0.7))

# protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
# protein.sequence = None
# protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8))

output = protein.sequence

  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
mistakes = 0
for i, (aa1, aa2) in enumerate(zip(output, sequence)):
    if aa1 != aa2:
        mistakes += 1

print(f"Original sequence: \n {sequence[:length]}")
print(f"Prompt: \n {prompt}")
print(f"Generated sequence: \n {output}")
print(f"Number of masked amino acids: {n_masked_aas}")
print(f"Number of mistakes: {mistakes}")
print(f"percentage of correct: {(1 - mistakes / n_masked_aas) * 100:.2f}%")

Original sequence: 
 MGSIQQDDVHNQIDHCNQSDDLPAARLNCNDVELFEVAGLACDETSSPTGMRDEMVLLSWLIALLRTREGGQIRYEWAYRYPEEEPVPRCLAMNEVVAGLQSSVKETAAAVSRHISADVSSPPAPASLLLSTSSLSQTSDEAKDEGLLHLEIAFENGLCKIRPTWHSENMLPFTVTRYARTLIDTVRLCISNCDAAIQDCLRPTAYDLDEIWRWNHNLPPTYNFCMHEIISDQAQKFPDKEAIASWDGSLTYRQIDQYSSFVARSLIGMGVGLHDVLPVCFEKSRWTIVAVLAVMKAGATFVLMDPTLPLARLQNMAQQVGAKMMVSSRGQYNLATEIIPNANVLVVEENTFSSLSAEQNGEPLPTVPSSALMYMIFTSGSTGTPKGVKISHETYTSSAIPRANAVGYTEDSRVLDFASYAFDVSIDSMLLTLGNGGCLCIPSDEDRLNDINGVIRRMKVNYAGLTPSVARILDADVISSLSGLGLGGEAVSARDVNLWGQDTRIIIGYGPCECTIGCTVNSSAATGRDYISIGPGNGAVIWIVDPNDHESLVPLGAVGELLVEGPIVGQGYLNDPEKTAAAFIEDPSWLVAGHEGYPGRRGRLYKTGDLGRYDPDGSGGIVFVGRKDTQVKLRGQRVELGEIESQLRARLPSETTVIAEVIVPQGSGGQPTLVAFVAAQTTKGHDHTGLEAAELPDELRRALSEADAELAKVLPRYMVPTAYIPVNHIPTLISGKTDRKRLRQFGATVDLRQLDQDATNTAARELSDLERRLRQAWSQTLKLQACSIRLQDNFFALGGDSLTAMKLVSVCRSQGLDLSVTSMFSNPTLSAMASVVRICDVDVQRTVPAFSMITSDMNSACVEAAEPCGVGPADIEDIYPCTPTQESLFTFSLKSVKPYVAQRVLCIPSHIDLNAWRKAWEDVVAALPILRTRVAQLQEPGLQQVVLKNSISWTQASDLAEYLENDRTQKMNLGESLAR

In [5]:
sequence = "MGVGLHDVLPVCFEKSRWTIVAVLAVMKAGATFVLMDPTLPLARLQNMAQQVGAKMMVSSRGQYNLATEIIPNANVLVVEENTFSSLSAEQNGEPLPTVPSSALMYMIFTSGSTGTPKGVKISHETYTSSAIPRANAVGYTEDSRVLDFASYAFDVSIDSMLLTLGNGGCLCIPSDEDRLNDINGVIRRMKVNYAGLTPSVARILDADVISSLSGLGLGGEAVSARDVNLWGQDTRIIIGYGPCECTIGCTVNSSAATGRDYISIGPGNGAVIWIVDPNDHESLVPLGAVGELLVEGPIVGQGYLNDPEKTAAAFIEDPSWLVAGHEGYPGRRGRLYKTGDLGRYDPDGSGGIVFVGRKDTQVKLRGQRVELGEIESQLRARLPSETTVIAEVIVPQGSGGQPTLVAFVAAQTTKGHDHTGLEAAELPDELRRALSEADAELAKVLPRYMVPTAYIPVNHIPTLISGKTDRKRLRQFGATVDLRQLDQDATNTAARELSDLERRLRQAWSQTLKLQACSIRLQDNFFALGGDSLTAMKLVSVCRSQGLDLSVTSMFSNPTLSAMASVVRICDVDVQRTVPAFSMITSDMNSACVEAAEPCGVGPADIEDIYPCTPTQESLFTFSLKSVKPYVAQRVLCIPSHIDLNAWRKAWEDVVAALPILRTRVAQLQEPGLQQVVLKNSISWTQASDLAEYLENDRTQKMNLGESLARYAIVEDSADGKRYMVWTIHHVLYDGWSEPIILKQVSDALQGQPVEVKAQMRDFVRFVRDSDDAAVQEFWRRELKGAVGPQFPRLPSRDFMPTPDALVERQVSLDTSSGSPFTMATLIRGAWALVASQYTGSDDIVFGETLTGRDIPLPGVESIVGPLIATVPIRVRILRGSTVESYLQAVQQSVLARTPYQHLGMQNIRKVSQDAQHACETGTGLVIQPEPEYVGSELGVERGDVVLEALHFNPYPLMLACGIRKGGFRVCASFDSSLIETRQMERMLAQLETACWQLSQGLSRKVDEISCLPEAELNQIWQW"
prompt = "MGVGLHDVLPVCFEKSRWTIVAVLAVMKAGATFVLMDPTLPLARLQNMAQQVGAKMMVSSRGQYNLATEIIPNANVLVVEENTFSSLSAEQNGEPLPTVPSSALMYMIFTSGSTGTPKGVKISHETYTSSAIPRANAVGYTEDSRVLDFASYAFDVSIDSMLLTLGNGGCLCIPSDEDRLNDINGVIRRMKVNYAGLTPSVARILDADVISSLSGLGLGGEAVSARDVNLWGQDTRIIIGYGPCECTIGCTVNSSAATGRDYISIGPGNGAVIWIVDPNDHESLVPLGAVGELLVEGPIVGQGYLNDPEKTAAAFIEDPSWLVAGHEGYPGRRGRLYKTGDLGRYDPDGSGGIVFVGRKDTQVKLRGQRVELGEIESQLRARLPSETTVIAEVIVPQGSGGQPTLVAFVAAQTTKGHDHTGLEAAELPDELRRALSEADAELAKVLPRYMVPTAYIPVNHIPTLISGKTDRKRLRQFGATVDLRQLDQDATNTAARELSDLERRLRQAWSQTLKLQACSIRLQDN__AL____LTAMKLVSVCRSQGLDLSVTSMFSNPTLSAMASVVRICDVDVQRTVPAFSMITSDMNSACVEAAEPCGVGPADIEDIYPCTPTQESLFTFSLKSVKPYVAQRVLCIPSHIDLNAWRKAWEDVVAALPILRTRVAQLQEPGLQQVVLKNSISWTQASDLAEYLENDRTQKMNLGESLARYAIVEDSADGKRYMVWTIHHVLYDGWSEPIILKQVSDALQGQPVEVKAQMRDFVRFVRDSDDAAVQEFWRRELKGAVGPQFPRLPSRDFMPTPDALVERQVSLDTSSGSPFTMATLIRGAWALVASQYTGSDDIVFGETLTGRDIPLPGVESIVGPLIATVPIRVRILRGSTVESYLQAVQQSVLARTPYQHLGMQNIRKVSQDAQHACETGTGLVIQPEPEYVGSELGVERGDVVLEALHFNPYPLMLACGIRKGGFRVCASFDSSLIETRQMERMLAQLETACWQLSQGLSRKVDEISCLPEAELNQIWQW"
length = len(prompt)
print(length)
n_masked_aas = prompt.count('_')
n_aas = len(prompt) - n_masked_aas

1024


In [3]:
# Generate the sequence.
protein = ESMProtein(sequence=prompt)
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=10, temperature=0.7))

output = protein.sequence

100%|██████████| 6/6 [01:12<00:00, 12.16s/it]
/home/kkj/ProtDiffusion/.venv/lib/python3.10/site-packages/esm/pretrained.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

In [6]:
mistakes = 0
for i, (aa1, aa2) in enumerate(zip(output, sequence)):
    if aa1 != aa2:
        mistakes += 1

print(f"Original sequence: \n {sequence[:length]}")
print(f"Prompt: \n {prompt}")
print(f"Generated sequence: \n {output}")
print(f"Number of masked amino acids: {n_masked_aas}")
print(f"Number of mistakes: {mistakes}")
print(f"percentage of correct: {(1 - mistakes / n_masked_aas) * 100:.2f}%")

Original sequence: 
 MGVGLHDVLPVCFEKSRWTIVAVLAVMKAGATFVLMDPTLPLARLQNMAQQVGAKMMVSSRGQYNLATEIIPNANVLVVEENTFSSLSAEQNGEPLPTVPSSALMYMIFTSGSTGTPKGVKISHETYTSSAIPRANAVGYTEDSRVLDFASYAFDVSIDSMLLTLGNGGCLCIPSDEDRLNDINGVIRRMKVNYAGLTPSVARILDADVISSLSGLGLGGEAVSARDVNLWGQDTRIIIGYGPCECTIGCTVNSSAATGRDYISIGPGNGAVIWIVDPNDHESLVPLGAVGELLVEGPIVGQGYLNDPEKTAAAFIEDPSWLVAGHEGYPGRRGRLYKTGDLGRYDPDGSGGIVFVGRKDTQVKLRGQRVELGEIESQLRARLPSETTVIAEVIVPQGSGGQPTLVAFVAAQTTKGHDHTGLEAAELPDELRRALSEADAELAKVLPRYMVPTAYIPVNHIPTLISGKTDRKRLRQFGATVDLRQLDQDATNTAARELSDLERRLRQAWSQTLKLQACSIRLQDNFFALGGDSLTAMKLVSVCRSQGLDLSVTSMFSNPTLSAMASVVRICDVDVQRTVPAFSMITSDMNSACVEAAEPCGVGPADIEDIYPCTPTQESLFTFSLKSVKPYVAQRVLCIPSHIDLNAWRKAWEDVVAALPILRTRVAQLQEPGLQQVVLKNSISWTQASDLAEYLENDRTQKMNLGESLARYAIVEDSADGKRYMVWTIHHVLYDGWSEPIILKQVSDALQGQPVEVKAQMRDFVRFVRDSDDAAVQEFWRRELKGAVGPQFPRLPSRDFMPTPDALVERQVSLDTSSGSPFTMATLIRGAWALVASQYTGSDDIVFGETLTGRDIPLPGVESIVGPLIATVPIRVRILRGSTVESYLQAVQQSVLARTPYQHLGMQNIRKVSQDAQHACETGTGLVIQPEPEYVGSELGVERGDVVLEALHFNPYPLMLACGIRKGGFRVCASFDSSL